In [ ]:
import ipywidgets as widgets
from IPython.display import display
import openai
import os

# Set OpenAI API key (ensure your environment variable is set or replace with your key)
openai.api_key = os.getenv("OPENAI_API_KEY",)

# Dropdowns & Input Fields for Patient Variables
ckd_stage = widgets.Dropdown(options=["I", "II", "IIIa", "IIIb", "IV", "V", "N/A"], description="CKD Stage:")
kidney_trend = widgets.Dropdown(options=["Stable", "Worsening", "Improving", "N/A"], description="Kidney Function:")
proteinuria_status = widgets.Dropdown(options=["Stable", "Worsening", "N/A"], description="Proteinuria:")
bp_status = widgets.Dropdown(options=["At Goal", "Above Goal", "N/A"], description="BP Status:")
bp_reading = widgets.Text(value="N/A", description="BP Reading:")
diabetes_status = widgets.Dropdown(options=["Controlled", "Uncontrolled", "N/A"], description="Diabetes:")
a1c_level = widgets.Text(value="N/A", description="A1c Level:")
fluid_status = widgets.Dropdown(options=["Normal", "Overloaded", "N/A"], description="Fluid Status:")
labs_review = widgets.Dropdown(options=["Reviewed and Stable", "Reviewed and Unstable", "Not Reviewed", "N/A"], description="Labs:")
potassium_level = widgets.Dropdown(options=["Normal", "Elevated", "Low", "N/A"], description="Potassium:")
bicarbonate_level = widgets.Dropdown(options=["Normal", "Low", "N/A"], description="Bicarbonate:")
hemoglobin_level = widgets.Dropdown(options=["Normal", "Low", "N/A"], description="Hemoglobin:")
iron_status = widgets.Dropdown(options=["Normal", "Iron Deficient", "N/A"], description="Iron Studies:")
pth_level = widgets.Dropdown(options=["Normal", "Elevated", "N/A"], description="PTH:")
vitamin_d_level = widgets.Dropdown(options=["Normal", "Low", "N/A"], description="Vitamin D:")
followup_appointment = widgets.Text(value="", description="Follow-up in:")

# Medication Change Option
med_change = widgets.Dropdown(options=["No", "Yes", "N/A"], description="Medication Change:")
med_change_types = widgets.SelectMultiple(
    options=["BP Medication", "Diabetes Medication", "Diuretic", "Potassium Binder", "Iron Supplement", "ESA Therapy", "Vitamin D Supplement", "Bicarbonate Supplement"],
    description="Med Changes:",
    disabled=True
)

def on_med_change(change):
    med_change_types.disabled = change["new"] != "Yes"

med_change.observe(on_med_change, names='value')

# Button to Generate AVS Response
generate_button = widgets.Button(description="Generate AVS")
output_area = widgets.Output()

# Editable text area to allow modifications after generation
generated_response_area = widgets.Textarea(
    value='',
    description='Editable AVS:',
    layout=widgets.Layout(width='100%', height='300px')
)

def generate_avs_response(b):
    # Map CKD stage to GFR ranges (for additional context in the prompt)
    gfr_ranges = {
        "I": "≥ 90 mL/min",
        "II": "89-60 mL/min",
        "IIIa": "59-45 mL/min",
        "IIIb": "44-30 mL/min",
        "IV": "29-15 mL/min",
        "V": "< 15 mL/min"
    }

    # Build the prompt string based on the input values
    prompt = "Generate a comprehensive, coherent AVS summary for the following patient details:\n\n"

    if ckd_stage.value != "N/A":
        prompt += f"- CKD Stage: {ckd_stage.value}"
        if ckd_stage.value in gfr_ranges:
            prompt += f" (GFR {gfr_ranges[ckd_stage.value]})"
        prompt += "\n"

    prompt += f"- Kidney Function Trend: {kidney_trend.value}\n"
    prompt += f"- Proteinuria: {proteinuria_status.value}\n"
    prompt += f"- Blood Pressure Status: {bp_status.value}\n"
    prompt += f"- Blood Pressure Reading: {bp_reading.value}\n"
    prompt += f"- Diabetes Status: {diabetes_status.value}\n"
    prompt += f"- A1c Level: {a1c_level.value}\n"
    prompt += f"- Fluid Status: {fluid_status.value}\n"
    prompt += f"- Labs Review: {labs_review.value}\n"
    prompt += f"  - Potassium Level: {potassium_level.value}\n"
    prompt += f"  - Bicarbonate Level: {bicarbonate_level.value}\n"
    prompt += f"  - Hemoglobin Level: {hemoglobin_level.value}\n"
    prompt += f"  - Iron Status: {iron_status.value}\n"
    prompt += f"  - PTH Level: {pth_level.value}\n"
    prompt += f"  - Vitamin D Level: {vitamin_d_level.value}\n"

    if followup_appointment.value:
        prompt += f"- Follow-up Appointment: In {followup_appointment.value}\n"

    prompt += f"- Medication Change: {med_change.value}\n"
    if med_change.value == "Yes" and med_change_types.value:
        prompt += f"  - Medication Changes: {', '.join(med_change_types.value)}\n"

    prompt += "\nPlease generate a concise yet detailed summary with recommendations, next steps, and any pertinent patient education points."

    try:
        # Call OpenAI ChatCompletion API
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or use "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are a knowledgeable medical assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=512,
            temperature=0.7  # Adjust for creativity
        )
        result = response.choices[0].message.content.strip()
    except Exception as e:
        result = f"Error during API call: {e}"

    with output_area:
        output_area.clear_output()
        # Populate the editable text area with the generated response
        generated_response_area.value = result
        display(generated_response_area)

# Bind Button Click Event
generate_button.on_click(generate_avs_response)

# Display Widgets
display(
    ckd_stage, kidney_trend, proteinuria_status, bp_status, bp_reading,
    diabetes_status, a1c_level, fluid_status, labs_review, potassium_level,
    bicarbonate_level, hemoglobin_level, iron_status, pth_level, vitamin_d_level,
    followup_appointment, med_change, med_change_types, generate_button, output_area
)
